In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib qt
%load_ext autoreload
%autoreload 2
import numpy as np
import time
import random
import torch
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))
import cvxpy as cp
from scipy.integrate import solve_ivp

# Example 1

In [2]:
A = np.array([[0,1],[-2,-1]])
B = np.array([[0],[1]])
G = np.array([[0],[1]])

C = np.array([[1,0]])
D = np.array([[0]])
print(A)
print(B)
print(C)
print(D)

[[ 0  1]
 [-2 -1]]
[[0]
 [1]]
[[1 0]]
[[0]]


In [17]:
n = 2
P = cp.Variable((n,n), PSD=True)
kappa = cp.Variable(1,nonneg=True)
M = kappa * np.array([[1,0],[0,-1]])
# M = cp.Variable((n,n), symmetric=True)
I = np.eye(2)
alpha = 0.1

tmp1 = cp.hstack((P@A+A.transpose()@P+2*alpha*P,P@B))
tmp2 = cp.hstack((B.transpose()@P,np.array([[0]])))
left = cp.vstack((tmp1,tmp2))
tmp3 = np.hstack((C,D))
tmp4 = np.hstack((C*0,D+1))
CDI = np.vstack((tmp3,tmp4))


constraints = []
constraints.append(P >> np.eye(2))
constraints.append(left + CDI.T@M@CDI << 0)
# constraints.append(left << 0)

prob = cp.Problem(cp.Minimize(1),constraints)
prob.solve()
print(prob.status)
print("The optimal value is", prob.value)
print("A solution P is")
print(P.value)
print(M.value)
print("---------------")
print(left.value)
print(CDI.T@M.value@CDI)
print(np.linalg.eig(left.value))

optimal
The optimal value is 1.0000000011910306
A solution P is
[[3.67269147 0.8990523 ]
 [0.8990523  1.82596211]]
[[ 2.35962578  0.        ]
 [ 0.         -2.35962578]]
---------------
[[-2.86167089 -0.6984746   0.8990523 ]
 [-0.6984746  -1.48862721  1.82596211]
 [ 0.8990523   1.82596211  0.        ]]
[[ 2.35962578  0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.         -2.35962578]]
(array([-3.88593931, -1.72506775,  1.26070896]), array([[-0.73338929, -0.67346254,  0.0926734 ],
       [-0.53405556,  0.65511767,  0.53442071],
       [ 0.42062431, -0.34244569,  0.84012269]]))


In [ ]:
def sin_fun(t,w) :
    return w*np.sin(t)
def constant(t,w) :
    return w
def model(t,x,w) :
    dx = A@np.expand_dims(x,1) + B*np.sin(x[0])+G*w(t)
    return np.squeeze(dx)

w_list = [6,0,-6]
t_list = []
y_list = []
for w in w_list :
    sol = solve_ivp(model,(0,20),np.array([2*np.pi,0]),args=([lambda t:constant(t,w)]),method='RK45',rtol=1e-6,atol=1e-10)
    t_list.append(sol.t)
    y_list.append(sol.y)
    
w = 6   
sol = solve_ivp(model,(0,20),np.array([2*np.pi,0]),args=([lambda t:sin_fun(t,w)]),method='RK45',rtol=1e-6,atol=1e-10)
t_sin = sol.t
y_sin = sol.y
w_sin = [sin_fun(t,w) for t in t_sin]

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
for t,y,w in zip(t_list,y_list,w_list) :
    plt.plot(t,y[0,:],label="w="+str(w))
plt.legend(fontsize=10)
plt.xlabel('t (s)',fontsize=15)
plt.ylabel('theta',fontsize=15)
plt.title('constant input',fontsize=15)
plt.subplot(122)
plt.plot(t_sin,y_sin[0,:])
plt.plot(t_sin,w_sin,'--',label="input")
plt.legend(fontsize=10)
plt.xlabel('t (s)',fontsize=15)
plt.ylabel('theta',fontsize=15)
plt.title('sinusoidal input',fontsize=15)
plt.savefig('Example1.eps')

# Example 2

In [ ]:
A = -1
B = -10
G = 1

C = 1
D = 0
print(A)
print(B)
print(C)
print(D)

In [ ]:
n = 1
P = cp.Variable((n,n), PSD=True)
kappa = cp.Variable(1,nonneg=True)
M = kappa * np.array([[0,1],[1,0]])
I = np.eye(n)
alpha = 1.0

tmp1 = cp.hstack((P*A+A*P+2*alpha*P,P*B))
tmp2 = cp.hstack((B*P,np.array([[0]])))
left = cp.vstack((tmp1,tmp2))
tmp3 = np.hstack((C,D))
tmp4 = np.hstack((C*0,D+1))
CDI = np.vstack((tmp3,tmp4))


constraints = []
constraints.append(P >> I)
constraints.append(left + CDI.T@M@CDI << 0)

prob = cp.Problem(cp.Minimize(1),constraints)
prob.solve()
print(prob.status)
print("The optimal value is", prob.value)
print("A solution P is")
print(P.value)
print(M.value)

In [ ]:
def sin_fun(t,w) :
    return w*np.sin(t*2)
def constant(t,w) :
    return w
def model(t,x,w) :
    dx = A*x + B*x*np.abs(x)+G*w(t)
    return np.squeeze(dx)

w_list = [5,10,15]
t_list = []
y_list = []
for w in w_list :
    sol = solve_ivp(model,(0,5),np.array([0]),args=([lambda t:constant(t,w)]),method='RK45',rtol=1e-6,atol=1e-10)
    t_list.append(sol.t)
    y_list.append(sol.y)
    
w = 3
sol = solve_ivp(model,(0,5),np.array([0]),args=([lambda t:sin_fun(t,w)]),method='RK45',rtol=1e-6,atol=1e-10)
t_sin = sol.t
y_sin = sol.y
w_sin = [sin_fun(t,w) for t in t_sin]

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
for t,y,w in zip(t_list,y_list,w_list) :
    plt.plot(t,y[0,:],label="w="+str(w))
plt.legend(fontsize=10)
plt.xlabel('t (s)',fontsize=15)
plt.ylabel('theta',fontsize=15)
plt.title('constant input',fontsize=15)
plt.subplot(122)
plt.plot(t_sin,y_sin[0,:])
plt.plot(t_sin,w_sin,'--',label="input")
plt.legend(fontsize=10)
plt.xlabel('t (s)',fontsize=15)
plt.ylabel('theta',fontsize=15)
plt.title('sinusoidal input',fontsize=15)
plt.savefig('Example2.eps')

# example 3

In [ ]:
A = np.array([[0,1],[0,-3]])
B = np.array([[0],[-1]])
G = np.array([[0],[1]])

C = np.array([[1,0]])
D = np.array([[0]])
print(A)
print(B)
print(C)
print(D)

In [ ]:
n = 2
P = cp.Variable((n,n), PSD=True)
kappa = cp.Variable(1,nonneg=True)
M = kappa * np.array([[-100,15],[15,-2]])
I = np.eye(2)
alpha = 0.95

tmp1 = cp.hstack((P@A+A.transpose()@P+2*alpha*P,P@B))
tmp2 = cp.hstack((B.transpose()@P,np.array([[0]])))
left = cp.vstack((tmp1,tmp2))
tmp3 = np.hstack((C,D))
tmp4 = np.hstack((C*0,D+1))
CDI = np.vstack((tmp3,tmp4))


constraints = []
constraints.append(P >> np.eye(2))
constraints.append(left + CDI.T@M@CDI << 0)

prob = cp.Problem(cp.Minimize(1),constraints)
prob.solve()
print(prob.status)
print("The optimal value is", prob.value)
print("A solution P is")
print(P.value)
print(M.value)

In [ ]:
def sin_fun(t,w) :
    return w*np.sin(t)
def constant(t,w) :
    return w
def nonlinear_spring(x) :
    if x >= 0 :
        ret = 5*x + 50 * x / (np.sqrt(100+x**2))
    else :
        ret = 10*x
    return ret
def model(t,x,w) :
    dx = A@np.expand_dims(x,1) + B*nonlinear_spring(x[0])+G*w(t)
    return np.squeeze(dx)

w_list = [60,0,-60]
t_list = []
y_list = []
for w in w_list :
    sol = solve_ivp(model,(0,20),np.array([-10,0]),args=([lambda t:constant(t,w)]),method='RK45',rtol=1e-6,atol=1e-10)
    t_list.append(sol.t)
    y_list.append(sol.y)
    
w = 60   
sol = solve_ivp(model,(0,20),np.array([-10,0]),args=([lambda t:sin_fun(t,w)]),method='RK45',rtol=1e-6,atol=1e-10)
t_sin = sol.t
y_sin = sol.y
w_sin = [sin_fun(t,w) for t in t_sin]

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
for t,y,w in zip(t_list,y_list,w_list) :
    plt.plot(t,y[0,:],label="w="+str(w))
plt.legend(fontsize=10)
plt.xlabel('t (s)',fontsize=15)
plt.ylabel('theta',fontsize=15)
plt.title('constant input',fontsize=15)
plt.subplot(122)
plt.plot(t_sin,y_sin[0,:])
plt.plot(t_sin,w_sin,'--',label="input")
plt.legend(fontsize=10)
plt.xlabel('t (s)',fontsize=15)
plt.ylabel('theta',fontsize=15)
plt.title('sinusoidal input',fontsize=15)
plt.savefig('Example3.eps')

# example 4

In [ ]:
A = np.array([[0,1],[-1,0]])
B = np.array([[0],[-1]])
G = np.array([[0],[1]])

C = np.array([[0,1]])
D = np.array([[0]])
print(A)
print(B)
print(C)
print(D)

In [ ]:
n = 2
P = cp.Variable((n,n), PSD=True)
M = cp.Variable((n,n), symmetric=True)
M11 = M[0,0]
M12 = M[0,1]
M22 = M[1,1]
# M11 = cp.Variable(1)
# M12 = cp.Variable(1)
# M22 = cp.Variable(1)
# M_tmp1 = cp.hstack((M11,M12))
# M_tmp2 = cp.hstack((M12,M22))
# M = cp.vstack((M_tmp1,M_tmp2))

I = np.eye(2)
alpha = 0.1

tmp1 = cp.hstack((P@A+A.transpose()@P+2*alpha*P,P@B))
tmp2 = cp.hstack((B.transpose()@P,np.array([[0]])))
left = cp.vstack((tmp1,tmp2))
tmp3 = np.hstack((C,D))
tmp4 = np.hstack((C*0,D+1))
CDI = np.vstack((tmp3,tmp4))

theta1 = 1
theta2 = 5

constraints = []
constraints.append(P >> I)
constraints.append(left + CDI.T@M@CDI << 0)
constraints.append(M11 + M12 * theta1 + theta1 * M12 + M22 * theta1 **2 >= 0)
constraints.append(M11 + M12 * theta2 + theta2 * M12 + M22 * theta2 **2 >= 0)
constraints.append(M22 <= 0)

prob = cp.Problem(cp.Minimize(1),constraints)
prob.solve()
print(prob.status)
print("The optimal value is", prob.value)
print("A solution P is")
print(P.value)
print(M.value)

In [ ]:
def sin_fun(t,w) :
    return w*np.sin(t)
def constant(t,w) :
    return w
def nonlinear_damper(x) :
    return 5*x - 64*x / np.sqrt(256+x**2)
def model(t,x,w) :
    dx = A@np.expand_dims(x,1) + B*nonlinear_damper(x[1])+G*w(t)
    return np.squeeze(dx)

w_list = [4,0,-4]
t_list = []
y_list = []
for w in w_list :
    sol = solve_ivp(model,(0,30),np.array([-10,0]),args=([lambda t:constant(t,w)]),method='RK45',rtol=1e-6,atol=1e-10)
    t_list.append(sol.t)
    y_list.append(sol.y)
    
w = 4
sol = solve_ivp(model,(0,30),np.array([-10,0]),args=([lambda t:sin_fun(t,w)]),method='RK45',rtol=1e-6,atol=1e-10)
t_sin = sol.t
y_sin = sol.y
w_sin = [sin_fun(t,w) for t in t_sin]

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
for t,y,w in zip(t_list,y_list,w_list) :
    plt.plot(t,y[0,:],label="w="+str(w))
plt.legend(fontsize=10)
plt.xlabel('t (s)',fontsize=15)
plt.ylabel('theta',fontsize=15)
plt.title('constant input',fontsize=15)
plt.subplot(122)
plt.plot(t_sin,y_sin[0,:])
plt.plot(t_sin,w_sin,'--',label="input")
plt.legend(fontsize=10)
plt.xlabel('t (s)',fontsize=15)
plt.ylabel('theta',fontsize=15)
plt.title('sinusoidal input',fontsize=15)
plt.savefig('Example4.eps')

# example 5

In [ ]:
A = np.array([[1,1],[0,1]])
B = np.array([[0],[1]])
B_p = np.array([[-1],[0]])

C = np.array([[1,0]])
D = np.array([[0]])
print(A)
print(B)
print(C)
print(D)

In [ ]:
S = cp.Variable((2,2), PSD=True)
L = cp.Variable((1,2))
K_p = cp.Variable(1)
# kappa = cp.Variable(1,nonneg=True)
kappa = 1
M = kappa * np.array([[0,1],[1,0]])
I = np.eye(n)
alpha = 1

constraints = []
constraints.append(A@S+S@A.T + B@L + L.T@B.T + 2*alpha*S << 0)
constraints.append(B_p + K_p*B + kappa * S@C.T ==  0)

prob = cp.Problem(cp.Minimize(1),constraints)
prob.solve()
print(prob.status)
print("The optimal value is", prob.value)
print("A solution P is")
print(S.value)
print(L.value)
print(K_p.value)

In [ ]:
P = np.linalg.inv(S.value)
print(P)
K_x = L.value@P
print(K_x)
K_p = K_p.value
print(K_p)

In [ ]:
def model(t,x) :
    p = x[0] * np.abs(x[0])
    u = K_x@np.expand_dims(x,1) + K_p * p
    dx = A@np.expand_dims(x,1) + B*u + B_p*p
    return np.squeeze(dx)
x0_list = [[10,10],[20,20],[30,30],[40,40],[50,50]]
t_list = []
y_list = []
for x0 in x0_list :
    sol = solve_ivp(model,(0,3),np.array(x0),method='RK45',rtol=1e-6,atol=1e-10)
    t_list.append(sol.t)
    y_list.append(sol.y)

In [ ]:
print_np(y_list[0])

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
for t,y in zip(t_list,y_list) :
    plt.plot(t,y[0,:])
plt.legend(fontsize=10)
plt.xlabel('t (s)',fontsize=15)
plt.ylabel('x1',fontsize=15)
# plt.title('x1',fontsize=15)
plt.grid(True)
plt.subplot(122)
for t,y in zip(t_list,y_list) :
    plt.plot(t,y[1,:])
plt.legend(fontsize=10)
plt.xlabel('t (s)',fontsize=15)
plt.ylabel('x2',fontsize=15)
# plt.title('x2',fontsize=15)
plt.grid(True)
plt.savefig('Example5.eps')